# **Ekploracyjna Analiza Danych (EDA)**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Cel Analizy**
1.  **Dystrybucja ocen ekspertów**
2.  **Dystrybucja długości raportów**
3.  **Powtarzalność wystąpień zestawów etykiet**

#### **Import Bibliotek**

In [21]:
import pandas as pd
import plotly.express as px
from collections import defaultdict

In [22]:
try:
  with open('data/data.jsonl', 'r') as f:
      df = pd.read_json(f, lines=True)
except KeyError:
  print("Plik z danymi nie został znaleziony. Proszę pamiętać, że nie jest on częścią tego repozytorium!")

#### **Kryteria (etykiety)**
* ##### **Kryterium 1** - Plan Transformacji Klimatycznej (ESRS E1-1)
* ##### **Kryterium 2** - Identyfikacja i Zarządzanie Ryzykami (ESRS E1-2)
* ##### **Kryterium 4** - Definicja Granic Konsolidacji (ESRS E1-6 Metodyka)
* ##### **Kryterium 6** - Raportowanie Danych Historycznych (ESRS 1 + ESRS E1)
* ##### **Kryterium 7** - Ujawnienie Wskaźników Intensywności (ESRS E1-6)
* ##### **Kryterium 8** - Wiarygodność Zdefiniowanych Celów (ESRS E1-4)

In [23]:
criterias = ['Kryterium 1', 'Kryterium 2', 'Kryterium 4', 'Kryterium 6', 'Kryterium 7', 'Kryterium 8']

## **Podstawowe informacje o danych**

##### **Język raportów:** Polski
##### **Raporty dotyczą lat:** 2021-2023

In [24]:
print(f"Informacje o dataframe:")
print(f"Ilość raportów (linii w zbiorze danych): {len(df)}")
print(f"Ilość wymiarów: {df.ndim}")
print(f"Kolumny: {df.columns}")

Informacje o dataframe:
Ilość raportów (linii w zbiorze danych): 393
Ilość wymiarów: 2
Kolumny: Index(['text', 'labels'], dtype='object')


## **Dystrybucja ocen ekspertów (etykiet)**

In [25]:
results_list = []

df_len = len(df)
for i in range(len(df['labels'][0])):
  count_ones = df[df['labels'].str[i] == 1]['labels'].count()
  count_zeros = df_len - count_ones
  row_data = {
      "Kryterium": criterias[i],
      "1": count_ones,
      "0": count_zeros
  }

  results_list.append(row_data)

data_labels = pd.DataFrame(results_list)

In [26]:
fig = px.bar(
  data_labels, 
  x=data_labels['Kryterium'], 
  y=[data_labels['1'], data_labels['0']],
  text_auto=True
)

fig.update_layout(
  xaxis_title="Kryteria",
  yaxis_title="",
  plot_bgcolor='white',
  legend_title_text = 'Etykiety'
)
fig.show()

### **Dystrybucja ocen ekspertów (etykiet) - wnioski:**

**Analiza**:
Na podstawie przeprowadzonej analizy, można stwierdzić że zbiór jest niezbalansowany pod kątem występowania klas, głównie dla kryteriów 2, 6 i 8, z przewagą klasy negatywnej (0).

**Wnioski**:

Podczas fine-tuningu modelu wymagane będzie ustawienie niestandardowych progów dla ewaluacji wyników.

Uzasadnione również będzie zastosowanie bardziej agresywnych technik optymalizacji funkcji straty np. Focal Loss.

## **Dystrybucja długości raportów**

In [27]:
results_list = []
for i in range(len(df['text'])):
  results_list.append(len(df['text'][i]))

In [28]:
fig = px.histogram(
  results_list, 
  x=results_list,
  text_auto=True,
  nbins=50
)

fig.update_layout(
  yaxis_title = "Ilość raportów",
  xaxis_title = "Ilość znaków",
  bargap=0.1
)

fig.show()

### **Dystrybucja długości raportów - wnioski:**

**Analiza**: Dystrybucja jest prawostronnie skośna, z większością raportów o długości od 0 do 400 000 znaków. Długi ogon rozkładu potwierdza obecność kilku bardzo obszernych raportów.

**Wnioski**: Dla procesu treningu modelu oznaczać to będzie konieczność podzielenia raportów na fragmenty oraz zastosowania technik przesuwnego okna podczas tokenizacji. Końcowo będzie to wymagało zastosowania odpowiedniej techniki agregacji wyników.

## **Powtarzalność wystąpień zestawów etykiet**

In [29]:
y = df['labels'].transform(tuple)
uniquesDict = defaultdict(int)

value_counts = y.value_counts()
top_5_series = value_counts.head(5)
bottom_5_series = value_counts.tail(5)

print("--- Top 5 najczęstszych zestawów etykiet ---")
print(top_5_series)

print("\n--- Bottom 5 najrzadszych zestawów etykiet ---")
print(bottom_5_series)

--- Top 5 najczęstszych zestawów etykiet ---
labels
(0, 0, 0, 0, 0, 0)    109
(1, 0, 0, 0, 0, 0)     47
(1, 1, 1, 1, 1, 0)     19
(1, 0, 1, 0, 1, 0)     18
(1, 1, 1, 1, 1, 1)     17
Name: count, dtype: int64

--- Bottom 5 najrzadszych zestawów etykiet ---
labels
(1, 1, 0, 0, 0, 1)    2
(1, 1, 0, 1, 1, 1)    2
(1, 1, 1, 1, 0, 0)    1
(1, 0, 0, 1, 0, 1)    1
(0, 0, 0, 0, 0, 1)    1
Name: count, dtype: int64


### **Powtarzalność wystąpień etykiet - wnioski:**

**Analiza**: Dominującym przypadkiem (109 wystąpień) jest zestaw (0, 0, 0, 0, 0, 0). Z drugiej strony spektrum znajduje się wiele kombinacji etykiet, które występują zaledwie raz lub dwa razy.

**Wnioski**:

W początkowym stadium prac należy unikać techniki undersamplingu. Ze względu na unikalny charakter każdego z raportów, usuwanie przykładów skrajnie negatywnych prowadziłoby do utraty cennego kontekstu.

Zarówno analiza dystrybucji pojedynczych etykiet, jak i całych zestawów, wskazuje na problem silnego niezbalansowania danych. Kluczowym elementem strategii powinno być zastosowanie zmodyfikowanej funkcji kosztu.